# Transport Layer Security (TLS)

## What to Focus On

- Using TLS adds a level of security that HTTP lacks.
- **Understand that TLS provides for secure message exchange over an unsecure channel**: start from the basis of what unsecure HTTP message transfer looks like
- **Learn that there are multiple aspects to security**: Learn what each of these services provide, and the particular problems each of them aims to address.

## The Transport Layer Security (TLS) Protocol

- The Transport Layer Security (TLS) protocol started life as a protocol called SSL (Secure Sockets Layer).
- The terms are often still used interchangeably
- There are three important security services that are provided by TLS: **Encryption**, **Authentication**, and **Integrity**. Each of these services are important in their own right, but when combined they provide for very secure message exchange over what is essentially an unsecure channel.<br><br>
- **Encryption**: a process of encoding a message so that it can only be read by those with an authorized means of decoding the message
- **Authentication**: a process to verify the identity of a particular party in the message exchange
- **Integrity**: a process to detect whether a message has been interfered with or faked<br><br>
- It isn't mandatory for an application which uses TLS that all three of these services are used simultaneously.  In practice however, all three services are generally used together to provide the most secure connection possible.

## TLS Encryption

- The way in which TLS sets up an encrypted connection is via a process known as the TLS Handshake.

### A Brief History of Cryptography

- **Cryptography**: the use of techniques for securing communication.

### Symmetric Key Encryption

- The underlying concept seen in the Vigenere cipher, whereby sender and receiver share a common encryption key, is still used in modern cryptographic systems. 
- A shared key system such as this is known as **symmetric key encryption**.

![](tls-encryption-symmetric.png)

### Asymmetric Key Encryption

- **Asymmetric key encryption**, also known as **public key encryption**, uses a pair of keys: a public key, and a private key.
- Unlike the symmetric system where the same key is used to encrypt and decrypt messages, in the asymmetric system the keys in the pair are non-identical: **the public key is used to encrypt and the private key to decrypt**.
- Messages encrypted with the public key can only be decrypted with the private key. 
- The public key is made openly available but the private key is kept in the sole possession of the message receiver.<br><br>
- Alice wants to receive encrypted messages. She generates a public key and a private key. 
- She makes the public key available but keeps the private key to herself. 
- Bob uses Alice's public key to encrypt a message and send it to Alice. Alice decrypts Bob's message using her private key.

![](tls-encryption-asymmetric.png)

- An important thing to note here is that this encryption is primarily intended to work in one direction. 
- Bob can send Alice messages encrypted with the public key which she can then decrypt with the private one. 
- The same key pair would not be used in the other direction for secure communication, since anyone with access to the public key can decrypt the message.

### The TLS Handshake

- To securely send messages via HTTP we want both the request and the response to be encrypted in such a way that they can only be decrypted by the intended recipient. 
- The most efficient way to do this is via symmetric key cryptography. If we want to use symmetric keys however, we also need a way to securely exchange the symmetric key.
- TLS uses a combination of symmetric and asymmetric cryptography. 
    - The bulk of the message exchange is conducted via symmetric key encryption
    - but the initial symmetric key exchange is conducted using asymmetric key encryption. 
- The process by which the initial secure connection is set up is conducted during what is known as the **TLS handshake**.<br><br>
- **TLS assumes TCP is being used at the Transport layer, and the TLS Handshake takes place after the TCP Handshake.**<br><br>
    1. The TLS Handshake begins with a `ClientHello` message which is sent immediately after the TCP `ACK`
        - This message contains the maximum version of the TLS protocol that the client can support, and a list of Cipher Suites that the client is able to use
    2. On receiving the `ClientHello` message, the server responds with a message of its own.  
        - This message includes a ServerHello, which sets the protocol version and Cipher Suite, as well as other related information. 
        - As part of this message the server also sends its certificate (which contains its public key), and a `ServerHelloDone` marker which indicates to the client that it has finished with this step of the handshake.
    3. Once the client has received the ServerHelloDone marker, it will initiate the key exchange process. 
        - It's this key exchange process that ultimately enables both the client and server to securely obtain a copy of the symmetric encryption key that will be used for the bulk of the secure message transfer between the two parties. 
        
![](tls-encryption-tls-handshake.png)

- The key points to remember about the TLS Handshake process is that it is used to:
    - Agree which version of TLS to be used in establishing a secure connection.
    - Agree on the various algorithms that will be included in the cipher suite.
    - Enable the exchange of symmetric keys that will be used for message encryption.<br><br>
- Something you should be aware of is that one of the implications of this complexity is its impact on performance. 
- The TLS handshake can add up to two round-trips of latency (depending on the TLS version) to the establishment of a connection between client and server prior to the point where any application data can be sent. 
- This is on top of the initial round trip resulting from the TCP Handshake.<br><br>
- Note: there is a protocol called **Datagram Transport Layer Security (DTLS)**, which is based on TLS. This protocol is specifically for use with network connections which use UDP rather than TCP at the Transport layer.

### Cipher Suites

- A **cipher** is a cryptographic algorithm; in other words they are sets of steps for performing encryption, decryption, and other related tasks. A cipher suite is a suite, or set, of ciphers.
- TLS uses different ciphers for different aspects of establishing and maintaining a secure connection.
- The algorithms for performing each of these tasks, when combined, form the cipher suite. 
- The suite to be used is agreed as part of the TLS Handshake. As part of the `ClientHello` message, the client sends a list of algorithms it supports for each required task, and the server chooses from these according to which algorithms it also supports.

## TLS Authentication

- During the TLS Handshake, as part of its response to the `ClientHello` message, the server provides its certificate.
    - part of the function of this certificate is so that the client can use the Public Key contained within it during the key exchange process. 
    - Another function of this certificate is to provide a means of identification for the party providing it.
- The exact way that the Public Key is used during this process varies depending on the Authentication algorithm selected as part of the Cipher Suite. Generally however, this process will be something along the following lines:

    - The server sends its certificate, which includes its public key.
    - The server creates a 'signature' in the form of some data encrypted with the server's private key.
    - The signature is transmitted in a message along with the original data from which the signature was created.
    - On receipt of the message, the client decrypts the signature using the server's public key and compares the decrypted data to the original version.
    - If the two versions match then the encrypted version could only have been created by a party in possession of the private key.

- Following a process such as this, we can identify that the server which provided the certificate during the initial part of the TLS Handshake is in possession of the private key, and therefore the actual owner of the certificate.

### Certificate Authorities and the Chain of Trust

- When it comes to digital certificates, the trustworthy sources are called Certificate Authorities (CAs).

<img src="tls-authentication-chain-of-trust.png" width=500>

- The purpose of this chain-like structure is the level of security it provides.

## TLS Integrity

- To add a further layer of security, TLS provides the functionality to check the integrity of data transported via the protocol.

### TLS Encapsulation

- The OSI model defines TLS as a Session layer protocol, and so existing in between Application layer (where HTTP resides) and the Transport layer (where TCP resides)
-  When it is transporting application data TLS encapsulates that data in the same way that we've seen with other Protocol Data Units. In other words, the data to be transported forms a payload, and meta data is attached in the form of header and trailer fields.

<img src="tls-integrity-tls-record-structure.png" width=500>

- **The Message Authentication Code is completely unrelated to the MAC Address (media access control address) discussed in an earlier lesson.**

### Message Authentication Code (MAC)

- The MAC field is similar in concept to the checksum fields we've already seen in other PDUs, although there is a difference in implementation as well as overall intention. 
    - The checksum field in, say, a TCP Segment is intended for error detection (i.e. to test if some data was corrupted during transport). 
    - The intention of the `MAC` field in a TLS record is to add a layer of security by providing a means of checking that the message hasn't been altered or tampered with in transit.
- The way this is implemented is through the use of a hashing algorithm:
    1. The sender will create what's called a digest of the data payload. This is effectively a small amount of data derived from the actual data that will be sent in the MAC field. The digest is created using a specific hashing algorithm combined with a pre-agreed hash value. This hashing algorithm to be used and hash value will have been agreed as part of the TLS Handshake process when the Cipher Suite is negotiated.
    2. The sender will then encrypt the data payload using the symmetric key (as described earlier in the Encryption section), encapsulate it into a TLS record, and pass this record down to the Transport layer to be sent to the other party.
    3. Upon receipt of the message, the receiver will decrypt the data payload using the symmetric key. The receiver will then also create a digest of the payload using the same algorithm and hash value. If the digest created by the receiver matches the digest received in the MAC field, this confirms the integrity of the message.